In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
filepath = 'D:/downloads/bike-sharing-demand/'

In [89]:
train = pd.read_csv(filepath+'train.csv',parse_dates=['datetime'])
test = pd.read_csv(filepath+'test.csv',parse_dates=['datetime'])
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    10886 non-null  datetime64[ns]
 1   season      10886 non-null  int64         
 2   holiday     10886 non-null  int64         
 3   workingday  10886 non-null  int64         
 4   weather     10886 non-null  int64         
 5   temp        10886 non-null  float64       
 6   atemp       10886 non-null  float64       
 7   humidity    10886 non-null  int64         
 8   windspeed   10886 non-null  float64       
 9   casual      10886 non-null  int64         
 10  registered  10886 non-null  int64         
 11  count       10886 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(8)
memory usage: 1020.7 KB


In [90]:
train_test = [train,test]

In [91]:
for dataset in train_test:
    dataset['year']=dataset.datetime.dt.year
    dataset['month']=dataset.datetime.dt.month
    dataset['day']=dataset.datetime.dt.day
    dataset['hour']=dataset.datetime.dt.hour
    dataset['minute']=dataset.datetime.dt.minute
    dataset['second']=dataset.datetime.dt.second
    dataset['dayofweek']=dataset.datetime.dt.dayofweek
    dataset.head(1)

In [8]:
test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,hour,minute,second,dayofweek
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,2011,1,20,0,0,0,3
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,1,0,0,3
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,2,0,0,3
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,0,0,3
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,4,0,0,3


In [11]:
train.shape # (10886, 19)
test.shape # (6493, 16)

(6493, 16)

Quiz1. windspeed 컬럼 0값을 풍속의 평균값으로 대체

In [14]:
for dataset in train_test:
    dataset.windspeed.replace(0,dataset.windspeed.mean(),inplace=True)

In [15]:
test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,hour,minute,second,dayofweek
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.002700,2011,1,20,0,0,0,3
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,12.631157,2011,1,20,1,0,0,3
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,12.631157,2011,1,20,2,0,0,3
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.001400,2011,1,20,3,0,0,3
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.001400,2011,1,20,4,0,0,3


In [92]:
trainWind0 = train[train.windspeed==0]
trainWindnot0 = train[train.windspeed!=0]

In [20]:
from sklearn.ensemble import RandomForestClassifier

In [93]:
def predictWindspeed(data):
    
    dataWind0 = data[data.windspeed==0]
    dataWindNot0 = data[data.windspeed!=0]
    
    wCol = ['season','weather','temp','atemp','humidity','year','month']
    # 나중에 회귀모델에서는 w1*season + w2*weather + w3*temp + ... + w7*month + bias
    
    dataWindNot0['windspeed'] = dataWindNot0['windspeed'].astype('str') # ??
    
    # RF 분류기
    rfModelWind = RandomForestClassifier()
    
    # 모델링
    rfModelWind.fit(dataWindNot0[wCol],dataWindNot0['windspeed'])
    
    # 예측
    wind0Values = rfModelWind.predict(dataWind0[wCol])
    
    predictWind0 = dataWind0.copy()
    predictWindNot0 = dataWindNot0.copy()
    
    # 예측 값 적용
    predictWind0['windspeed'] = wind0Values
    
    data = predictWindNot0.append(predictWind0)
    data.reset_index(inplace=True)
    data.drop(['index'],axis=1)
    
    return data

train = predictWindspeed(train)
#test = predictWindspeed(test)

<ipython-input-93-572fac1a2951>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataWindNot0['windspeed'] = dataWindNot0['windspeed'].astype('str') # ??


In [26]:
train.windspeed.isnull().sum()

0

In [27]:
test.windspeed.isnull().sum()

0

In [29]:
train.windspeed.describe()

count      10886
unique        27
top       7.0015
freq        1304
Name: windspeed, dtype: object

In [94]:
# 연속형 -> 범주형 
category_fn = ['season','holiday','workingday','weather','year','month','hour','dayofweek']

for v in category_fn:
    train[v] = train[v].astype('category')
    test[v] = test[v].astype('category')
    
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   index       10886 non-null  int64         
 1   datetime    10886 non-null  datetime64[ns]
 2   season      10886 non-null  category      
 3   holiday     10886 non-null  category      
 4   workingday  10886 non-null  category      
 5   weather     10886 non-null  category      
 6   temp        10886 non-null  float64       
 7   atemp       10886 non-null  float64       
 8   humidity    10886 non-null  int64         
 9   windspeed   10886 non-null  object        
 10  casual      10886 non-null  int64         
 11  registered  10886 non-null  int64         
 12  count       10886 non-null  int64         
 13  year        10886 non-null  category      
 14  month       10886 non-null  category      
 15  day         10886 non-null  int64         
 16  hour        10886 non-

In [32]:
from sklearn.metrics import make_scorer

In [33]:
def rmsle(pv, av):  # predict value , actual value
    
    pv = np.array(pv)
    av = np.array(av)
    
    log_predict = np.log(pv+1)
    log_actual = np.log(av+1)
    
    res = log_predict-log_actual
    res = np.square(res)
    res = np.mean(res)
    score = np.sqrt(res)
    return score

In [34]:
rmsle_scorer = make_scorer(rmsle)
rmsle_scorer

make_scorer(rmsle)

### Cross Validation (교차 검증)  
 - 목적 : 일반화 성능 측정  
__K-Fold__  
- 데이터를 여러 겹(fold)으로 나누고 , 트레이닝/테스트 용으로 나눈 폴드를 다양하게 적용하여 모델을 학습하고 평가
- 각 모델의 정확도의 평균을 최종 모델 일반화 성능


In [35]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [53]:
kfold = KFold(n_splits=2, shuffle=True , random_state=42)

In [130]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=500,random_state=42)

In [142]:
fn = ['season','holiday','workingday',
      'weather','hour', 'year',
      'dayofweek','temp','atemp','humidity','windspeed']
fn

['season',
 'holiday',
 'workingday',
 'weather',
 'hour',
 'year',
 'dayofweek',
 'temp',
 'atemp',
 'humidity',
 'windspeed']

In [143]:
xTrain = train[fn]
yTrain = train['count']
xTrain.shape

(10886, 11)

In [144]:
xTest = test[fn]
xTest.shape

(6493, 11)

In [145]:
xTrain.windspeed = xTrain.windspeed.astype(np.float64)
xTest.windspeed = xTest.windspeed.astype(np.float64)

D:\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [146]:
model = rf.fit(xTrain,yTrain)

In [147]:
prediction = model.predict(xTest)

In [148]:
prediction

array([ 11.876     ,   5.082     ,   4.01066667, ...,  99.22466667,
       101.13633333,  46.70466667])

In [149]:
score = cross_val_score(model,xTrain,yTrain,cv=kfold,scoring=rmsle_scorer)
score

array([0.35415769, 0.36543885])

In [150]:
test['count'] = prediction
result = test[['datetime','count']]
result.to_csv('rf5.csv',index=False)
print('File Saved')

File Saved
